<div class="alert alert-block alert-info">
⚠️All notebooks should start with the following cell ⚠️.
<br>
<br>
This will ensure that the notebook is set in the correct working directory and that all internal modules can be loaded without setting an entry point.
</div>

In [2]:
from utils import setup_env_path

setup_env_path()

%load_ext autoreload
%autoreload 2

Former working directory:  /home/juldpnt/APT/fil-rouge-pollinisateurs/notebooks
Current working directory:  /home/juldpnt/APT/fil-rouge-pollinisateurs


<div class="alert alert-block alert-danger">
Put your API key in the following cell.<br><br>
⚠️But DO NOT commit it to the repository, as it is a secret.⚠️
</div>

In [3]:
API_key = '2b10Me1HF0rfjoGWCseolNa8e'

# Plantnet use case - Data quality

## Data import 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('data/spipoll.csv')


/tmp/ipykernel_1011/3851440610.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/spipoll.csv')


## Data aggregation

In [8]:
values_to_check = ["Je ne sais pas", "Plante inconnue"]


df_filtered = df.loc[
    (
        (df["plante_sc"].isna() | df["plante_sc"].isin(values_to_check))
        & (df["plante_fr"].isna() | df["plante_fr"].isin(values_to_check))
        & (df["plante_precision"].isna() | df["plante_precision"].isin(values_to_check))
    )
    | ((df["plante_inconnue"] == 1.0))
]
df_filtered.shape

(18249, 31)

In [9]:
# group by collection_id and count the number of rows
missing_plants = (df_filtered
                    .drop_duplicates(subset='collection_id', keep='first')
                    )
missing_plants.shape
print(f"{(df_filtered.shape[0]/df.shape[0]*100):.2f} % of the data is missing all plant information")

2.72 % of the data is missing all plant information


## Get data from API

In [12]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor
predictor = PlantNetPredictor(key=API_key)

In [15]:
missing_plants['photo_fleur'].iloc[0]

'https://spgp-api.65mo.fr/api/containers/spgp/download/1546962826333.jpg'

In [16]:
missing_plants.head()

,collection_id,protocole_long,user_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,...,insecte_fr,insecte_denominationPlusPrecise,insecte_CdNomtaxref,insecte_abondance,insecte_commentaire,insecte_vu_sur_fleur,nb_validation,nb_suggestion,latitude,longitude
46,14,NaN,11 784,Plante inconnue,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,Les Mouches difficiles à déterminer,NaN,NaN,1,NaN,0.0,3.0,NaN,44.694536,4.662742
364,117,NaN,11 784,Plante inconnue,NaN,NaN,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,Les Mouches aux reflets métalliques,NaN,NaN,1,NaN,0.0,3.0,NaN,44.729200,4.574164
574,177,NaN,11 784,Plante inconnue,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,Les Mouches difficiles à déterminer,NaN,NaN,1,NaN,0.0,3.0,NaN,44.719394,4.598397
730,215,NaN,8 868,NaN,NaN,NaN,1.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,L'Abeille mellifère,NaN,NaN,1,NaN,0.0,3.0,NaN,42.674231,9.291175
731,216,NaN,8 868,NaN,NaN,NaN,1.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,L'Abeille mellifère,NaN,NaN,1,NaN,0.0,3.0,NaN,42.674242,9.291344


In [26]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor

predictor = PlantNetPredictor(key = API_key)
prediction = None

try:
    prediction = predictor.predict(imageURL= missing_plants['photo_fleur'].iloc[-1], 
                    organs = "auto",
                    includeRelatedImages=False)
except Exception as e:
    print(f"Error: {e}")

In [32]:
prediction["results"][0]["species"]["family"]["scientificNameWithoutAuthor"]

'Asteraceae'

In [27]:
prediction

{'query': {'project': 'weurope',
  'images': ['https://spgp-api.65mo.fr/api/containers/spgp/download/65a9ef74-7840-451b-bd93-8677a52adcd7.jpg'],
  'organs': ['auto'],
  'includeRelatedImages': False,
  'noReject': False},
 'language': 'fr',
 'preferedReferential': 'weurope',
 'results': [{'score': 0.14201,
   'species': {'scientificNameWithoutAuthor': 'Tolpis virgata',
    'scientificNameAuthorship': 'Bertol.',
    'genus': {'scientificNameWithoutAuthor': 'Tolpis',
     'scientificNameAuthorship': 'Adanson',
     'scientificName': 'Tolpis Adanson'},
    'family': {'scientificNameWithoutAuthor': 'Asteraceae',
     'scientificNameAuthorship': '',
     'scientificName': 'Asteraceae'},
    'commonNames': ['Trépane effilée'],
    'scientificName': 'Tolpis virgata Bertol.'},
   'gbif': {'id': '3097191'}},
  {'score': 0.1416,
   'species': {'scientificNameWithoutAuthor': 'Crepis micrantha',
    'scientificNameAuthorship': 'Czerep.',
    'genus': {'scientificNameWithoutAuthor': 'Crepis',
     